In [ ]:
drive_dir1 = '/content/drive/MyDrive/datathon2024/Atlanta_supply_dat_UC_buildings.csv'
drive_dir2 = '/content/drive/MyDrive/datathon2024/More_Atlanta_Supply.csv'
import pandas as pd
import math

# Read the CSV file into a DataFrame, then merge along rows
df1 = pd.read_csv(drive_dir1)
df2 = pd.read_csv(drive_dir2)

df = pd.concat([df1, df2], ignore_index=True, sort=False)
df=df.dropna(subset=['Size_sf', 'YearQuarterGroundBroken'])

# Define a function to calculate TimeCompleted based on Size_sf
def calculate_time_completed(row):
    if row['Size_sf'] < 99999:
        quarters_to_add = 2
    elif 100000 < row['Size_sf'] < 299999:
        quarters_to_add = 3
    elif 300000 < row['Size_sf'] < 599999:
        quarters_to_add = 4
    elif 600000 < row['Size_sf'] < 999999:
        quarters_to_add = 5
    else:
        quarters_to_add = 6

    year, quarter = divmod(row['YearQuarterGroundBroken'], 1)
    quarter += quarters_to_add
    year += math.floor(quarter / 4)
    quarter %= 4
    if quarter == 0:
        quarter = 4
    return year + quarter  # Format as year.quarter
# Apply the function to create the TimeCompleted column
df['completions_sf'] = df.apply(calculate_time_completed, axis=1)


def fin_in_four(row):

    if row['Size_sf'] < 99999:
        quarters_to_add = 2
    elif 100000 < row['Size_sf'] < 299999:
        quarters_to_add = 3
    elif 300000 < row['Size_sf'] < 599999:
        quarters_to_add = 4
    elif 600000 < row['Size_sf'] < 999999:
        quarters_to_add = 5
    else:
        quarters_to_add = 6

    if quarters_to_add <= 5:
        fin_in_four_quarters = True
    else:
        fin_in_four_quarters = False
    return fin_in_four_quarters
df['fin_in_four_quarters'] = df.apply(fin_in_four, axis=1)
# Print the updated DataFrame
print(df[['completions_sf', 'YearQuarterGroundBroken']], sum(df['fin_in_four_quarters']))
#df.drop(['fin_in_four_quarters'], axis=1)



     completions_sf  YearQuarterGroundBroken
0            2026.1                   2024.1
1            2025.2                   2023.2
2            2026.3                   2023.3
3            2026.4                   2023.4
4            2026.2                   2023.2
..              ...                      ...
672          2026.1                   2023.1
673          2026.1                   2024.1
674          2026.2                   2023.2
675          2024.4                   2022.4
676          2027.1                   2024.1

[676 rows x 2 columns] 638


In [ ]:
drive_dir = '/content/drive/MyDrive/datathon2024/Atlanta_supply_dat_stock_data.csv'

d_f = pd.read_csv(drive_dir)
d_f['stock_SF'] = d_f['Stock_sf'] + d_f['Completions_sf']

d_f['Stock_Under_Construction_sf'] = 0
d_f['under_construction_as_a_percentage_of_Stock'] = 0

# Print the updated DataFrame
print(d_f)

   MarketCode MarketName  YearQuarter  Stock_sf  Completions_sf  \
0      ATLANT    Atlanta       2000.1    397465            4394   
1      ATLANT    Atlanta       2000.2    401081            3616   
2      ATLANT    Atlanta       2000.3    405850            4769   
3      ATLANT    Atlanta       2000.4    412672            6822   
4      ATLANT    Atlanta       2001.1    416867            4195   
..        ...        ...          ...       ...             ...   
91     ATLANT    Atlanta       2022.4    696700            9045   
92     ATLANT    Atlanta       2023.1    703240            6540   
93     ATLANT    Atlanta       2023.2    708037            4797   
94     ATLANT    Atlanta       2023.3    717677            9640   
95     ATLANT    Atlanta       2023.4    730346           12669   

    Stock_Under_Construction_sf  under_construction_as_a_percentage_of_Stock  \
0                             0                                            0   
1                             0    

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error

# Assuming df is your DataFrame with columns including sq ft, stage completed, and output quarters
# If df is not defined, load or create your DataFrame first

image_paths = []
image_labels = []
for data_dir, label in zip(data_dirs, labels):
    folder_path = os.path.join(drive_dir, data_dir)
    images = os.listdir(folder_path)
    for image in images:
        image_paths.append(os.path.join(folder_path, image))
        image_labels.append(label)
        list = image.split("_")
        #print(list)
        Latitude = []
        Longitude = []

        for i in list:
        # Extract the numerical parts and convert them to float
        latitude = float(list[1])
        # Extract the numerical part of the entry in list[2] for longitude
        longitude = float(list[2].split('.')[0] + '.' + list[2].split('.')[1])  # Assuming the numerical part is before the dot

        # Append the numerical parts to the respective lists
        Latitude.append(latitude)
        Longitude.append(longitude)

        for keys, values in df["Latitude"].items():
            if Latitude is in (df["Latitude"].values) and Longitude is in (df["Longitude"].values):
                df['stage_completed'] = label

print(Latitude, Longitude)




X = df[['stock_SF', 'stage_completed']]
y = df['QuartersTrackedInDatabase']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define preprocessing steps for different types of features
numeric_features = ['stock_SF']
categorical_features = ['stage_completed']

numeric_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])

# Create a pipeline with preprocessing and linear regression model
model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', LinearRegression())
])

# Fit the model
model.fit(X_train, y_train)

# Make predictions
y_pred = model.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
print(f'Mean Squared Error: {mse}')


[33.9960678] [-83.9604705]


"\nX = df[['stock_SF', 'stage_completed']]  \ny = df['QuartersTrackedInDatabase']  \n\n# Split the data into training and testing sets\nX_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)\n\n# Define preprocessing steps for different types of features\nnumeric_features = ['stock_SF']\ncategorical_features = ['stage_completed']\n\nnumeric_transformer = Pipeline(steps=[\n    ('scaler', StandardScaler())\n])\n\ncategorical_transformer = Pipeline(steps=[\n    ('onehot', OneHotEncoder(handle_unknown='ignore'))\n])\n\npreprocessor = ColumnTransformer(\n    transformers=[\n        ('num', numeric_transformer, numeric_features),\n        ('cat', categorical_transformer, categorical_features)\n    ])\n\n# Create a pipeline with preprocessing and linear regression model\nmodel = Pipeline(steps=[\n    ('preprocessor', preprocessor),\n    ('regressor', LinearRegression())\n])\n\n# Fit the model\nmodel.fit(X_train, y_train)\n\n# Make predictions\ny_pred = model